# **XOX Oyunu Gym**
---

Başlangıçta tüm kodu çalıştırmak ve Google Colab kullanarak eğitmek için gerekli kurulumları yapıyoruz





### Stable-baselines Kurulumu

Linux veya Google Colab için

In [1]:
#@title Kurulum (değiştirmeyin)
!pip install stable-baselines3[extra,tests,docs]>=0.11.0a4
!pip install git+https://github.com/DLR-RM/stable-baselines3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.
datascience 0.10.6 requires coverage==3.7.1, but you have coverage 6.4.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
coveralls 0.5 requires coverage<3.999,>=3.6, but you have coverage 6.4.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-d8p7u3ny
  Running command git clone -q https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-d8p7u3ny
  Created wheel for stable-baselines3: filename=stable_baselines3-1.5.1a8-py3-none-any.whl size=165288 sha256=2e931a1c0889ea083b3b947edd7b76c3b640e712dea249c5dc46b52237

### RLBaselinesZoo Kurulumu

In [2]:
#@title RLBaselinesZoo'yu Yükleme (değiştirmeyin)

IN_COLAB = False

if IN_COLAB:
    !git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo
    !cd rl-baselines3-zoo/
    !apt-get install swig cmake ffmpeg
    !pip install -r /content/rl-baselines3-zoo/requirements.txt

## Kodun gerçekten başladığı yer burasıdır.
---
### İçe aktarma/başlatma işlemleri

In [3]:
import os
from subprocess import Popen, PIPE

import numpy as np
import matplotlib.pyplot as plt

import gym
from gym import spaces
from gym.spaces import Discrete, Box


from stable_baselines3 import DQN, PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.env_util import make_vec_env

os.makedirs('logs', exist_ok=True)

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

cwd = os.getcwd()

%matplotlib inline
%load_ext tensorboard

### Özneyi özel politikalarla çalıştırma
---
Bundan sonraki adım oyunun sahip olduğu tüm kurallarla çevreyi (environment) oluşturmak.


In [4]:
from gym.spaces import Dict, Discrete, Box, Tuple
from gym import spaces
import random
lista=[0,1,2,3,4,5,6,7,8]
class MyEnv(gym.Env):

  metadata = {'render.modes': ['console']}
  # Definimos las constantes
  # Cada posicion representa un lugar de un vector de una dimensión
  POSICION0=0
  POSICION1=1
  POSICION2=2
  POSICION3=3
  POSICION4=4
  POSICION5=5
  POSICION6=6
  POSICION7=7
  POSICION8=8



  def __init__(self):
    # Para saber si jugó cierta posición o no
    self.JUGADO0=False
    self.JUGADO1=False
    self.JUGADO2=False
    self.JUGADO3=False
    self.JUGADO4=False
    self.JUGADO5=False
    self.JUGADO6=False
    self.JUGADO7=False
    self.JUGADO8=False
    # Si es el turno o no del adversario
    self.turno=True
    # Lista para jugar un número random, para simular un adversario
    self.lista=[0,1,2,3,4,5,6,7,8]
    self.action_space = Discrete(9)
    self.observation_space = Box(low=np.zeros(9), high=np.ones(9))
    # estado inicial, 0 nadie jugó, 1 jugó el CPU y 2 el adversario simulado
    self.state = [0,0,0,0,0,0,0,0,0]
    

  def reset(self):
    # Reinicio los estados para poner puede jugar de nuevo
    self.JUGADO0=False
    self.JUGADO1=False
    self.JUGADO2=False
    self.JUGADO3=False
    self.JUGADO4=False
    self.JUGADO5=False
    self.JUGADO6=False
    self.JUGADO7=False
    self.JUGADO8=False
    # seteo el estado todo en cero de nuevo
    self.state=np.zeros(9)

    # acá simulo que al comienzo 1 vez si y una vez no, un adversario
    # imaginario juegue random un movimiento
    if self.turno:
      self.jugar_random()

    self.turno= not self.turno
    return np.array([self.state]).astype(np.float32)

  def jugar_random(self):
    # Acá simulo una posicion random, mirando si ya tiene algo la casilla,
    # en donde juega, sino, juega otra y así
    indices=[0,1,2,3,4,5,6,7,8]
    random.shuffle(indices)
    indices
    for index in indices:
      if self.state[index]==0:
        self.state[index]=2
        if index==0:
          self.JUGADO0==True
        if index==1:
          self.JUGADO1==True
        if index==2:
          self.JUGADO2==True
        if index==3:
          self.JUGADO3==True
        if index==4:
          self.JUGADO4==True
        if index==5:
          self.JUGADO5==True
        if index==6:
          self.JUGADO6==True
        if index==7:
          self.JUGADO7==True
        if index==8:
          self.JUGADO8==True


        break

  def step(self, action):
    # Me fijo la accion que ejecuta, y si no es su su turno y si la está 
    # ocupada la casilla por el adversario lo penalizo
    reward=0
    if action==0 and self.state[0]==2 and self.JUGADO0==False:
      self.JUGADO0==True
      reward= reward-100
    if action==1  and self.state[1]==2 and self.JUGADO1==False:
      self.JUGADO1==True
      reward= reward-100
    if action==2 and self.state[2]==2 and self.JUGADO2==False:
      self.JUGADO2==True
      reward= reward-100
    if action==3 and self.state[3]==2 and self.JUGADO3==False:
      self.JUGADO3==True
      reward= reward-100
    if action==4 and self.state[4]==2 and self.JUGADO4==False:
      self.JUGADO4==True
      reward= reward-100
    if action==5 and self.state[5]==2 and self.JUGADO5==False:
      self.JUGADO5==True
      reward= reward-100
    if action==6 and self.state[6]==2 and self.JUGADO6==False:
      self.JUGADO6==True
      reward= reward-100
    if action==7 and self.state[7]==2 and self.JUGADO7==False:
      self.JUGADO7==True
      reward= reward-100
    if action==8 and self.state[8]==2 and self.JUGADO8==False:
      self.JUGADO8==True
      reward= reward-100

    # si juega bien que se actualize el estado y que juege random el adversario
    if action == self.POSICION0 and self.state[0]==0 and self.JUGADO0==False:
      self.state[0] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO0=True
    if action == self.POSICION1 and self.state[1]==0 and self.JUGADO1==False:
      self.state[1] = 1
      if not self.check_win():
        self.jugar_random()    
      self.JUGADO1=True
    if action == self.POSICION2 and self.state[2]==0 and self.JUGADO2==False:
      self.state[2] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO2=True
    if action == self.POSICION3 and self.state[3]==0 and self.JUGADO3==False:
      self.state[3] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO3=True
    if action == self.POSICION4 and self.state[4]==0 and self.JUGADO4==False:
      self.state[4] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO4=True
    if action == self.POSICION5 and self.state[5]==0 and self.JUGADO5==False:
      self.state[5] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO5=True
    if action == self.POSICION6 and self.state[6]==0 and self.JUGADO6==False:
      self.state[6] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO6=True
    if action == self.POSICION7 and self.state[7]==0 and self.JUGADO7==False:
      self.state[7] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO7=True
    if action == self.POSICION8 and self.state[8]==0 and self.JUGADO8==False:
      self.state[8] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO8=True


    # si ya jugó y juega de nuevo el mismo lugar, lo penalizo, para que aprenda
  
    if self.JUGADO0 and action==0:
      reward = reward-100
    if self.JUGADO1 and action==1:
      reward = reward-100
    if self.JUGADO2 and action==2:
      reward = reward-100
    if self.JUGADO3 and action==3:
      reward = reward-100
    if self.JUGADO4 and action==4:
      reward = reward-100
    if self.JUGADO5 and action==5:
      reward = reward-100
    if self.JUGADO6 and action==6:
      reward = reward-100
    if self.JUGADO7 and action==7:
      reward = reward-100
    if self.JUGADO8 and action==8:
      reward = reward-100

    #Si gana lo recompensamos
    if self.check_win():
      reward = reward + 1000
      done=True
    else: 
      pass
    # Si pierde lo penalizamos
    if self.check_lose():
      reward = reward -1000
      done=True
    else: 
      pass

    # Si empata, lo penalizo, para que aprenda que esta mal
    if self.check_draw():
      reward = reward -100
      done=True
    else: 
      pass

    # me dijo los estados para terminar o no
    if self.check_win() or self.check_win() or self.check_draw():
      done= True
    else:
      done=False


    info = {}

    return np.array([self.state]).astype(np.float32), reward, done, info

  def check_win(self):
    #filas
    if self.state[0]==1 and self.state[1]==1 and self.state[2]==1:
      return True
    if self.state[3]==1 and self.state[4]==1 and self.state[5]==1:
      return True
    if self.state[6]==1 and self.state[7]==1 and self.state[8]==1:
      return True
    #columnas
    if self.state[0]==1 and self.state[3]==1 and self.state[6]==1:
      return True
    if self.state[1]==1 and self.state[4]==1 and self.state[7]==1:
      return True
    if self.state[2]==1 and self.state[5]==1 and self.state[8]==1:
      return True
    #diagonal
    if self.state[0]==1 and self.state[4]==1 and self.state[8]==1:
      return True
    if self.state[2]==1 and self.state[4]==1 and self.state[6]==1:
      return True
    else:
      return False

  def check_lose(self):
    #filas
    if self.state[0]==2 and self.state[1]==2 and self.state[2]==2:
      return True
    if self.state[3]==2 and self.state[4]==2 and self.state[5]==2:
      return True
    if self.state[6]==2 and self.state[7]==2 and self.state[8]==2:
      return True
    #columnas
    if self.state[0]==2 and self.state[3]==2 and self.state[6]==2:
      return True
    if self.state[1]==2 and self.state[4]==2 and self.state[7]==2:
      return True
    if self.state[2]==2 and self.state[5]==2 and self.state[8]==2:
      return True
    #diagonal
    if self.state[0]==2 and self.state[4]==2 and self.state[8]==2:
      return True
    if self.state[2]==2 and self.state[4]==2 and self.state[6]==2:
      return True
    else:
      return False
    
  def check_draw(self):
    draw = False
    contador=0
    for i in [0,1,2,3,4,5,6,7,8]:
      if self.state[i]!=0:
        contador +=1
    
    if contador==9:
      draw = True
    
    return draw

  def render(self, mode='console'):
    if mode != 'console':
      raise NotImplementedError()
    # en nuestra interfaz de consola, representamos el agente como una cruz, y 
    # el resto como un punto
    for i in range(0,9,3):
      if i==0:
        print(f"{self.state[i]} {self.state[i+1]} {self.state[i+2]} ")
      if i==3:
        print(f"{self.state[i]} {self.state[i+1]} {self.state[i+2]} ")
      if i==6:
        print(f"{self.state[i]} {self.state[i+1]} {self.state[i+2]} ")

  def close(self):
    pass

## Modelin eğitimine devam ediyoruz
---


In [5]:
env = MyEnv()
env = make_vec_env(lambda: env, n_envs=1)
model = PPO('MlpPolicy', env, verbose=1).learn(5000)

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


Using cpu device
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 10.4      |
|    ep_rew_mean     | -2.97e+03 |
| time/              |           |
|    fps             | 1456      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 10.8         |
|    ep_rew_mean          | -3.68e+03    |
| time/                   |              |
|    fps                  | 1082         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0001672678 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.2         |
|    explained_variance   | 1.9

Daha sonra PyGame ile yapılan oyunda kullanacağım eğitilmiş modeli kaydetmeye devam ediyorum.

In [6]:
model.save("PPO5millonesFinal ")
# para cargarmodel = PPO.load("model5millonesAmbos")

## Eğitilmiş özneye bakalım
---
Burada, öznenin 100'den fazla eylemi nasıl gerçekleştirdiğinin küçük bir gösterimini yapıyoruz.

In [7]:
env=MyEnv()
obs = env.reset()
for i in range(100):
    action, _states = model.predict(obs)
    print(action)
    obs, rewards, dones, info = env.step(action)
    env.render(mode='console')
    #print(dones)
    if dones:
        obs = env.reset()
    print("-----------")

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


[4]
0.0 0.0 0.0 
0.0 2.0 0.0 
0.0 0.0 0.0 
-----------
[3]
2.0 0.0 0.0 
1.0 2.0 0.0 
0.0 0.0 0.0 
-----------
[5]
2.0 2.0 0.0 
1.0 2.0 1.0 
0.0 0.0 0.0 
-----------
[2]
2.0 2.0 1.0 
1.0 2.0 1.0 
2.0 0.0 0.0 
-----------
[0]
2.0 2.0 1.0 
1.0 2.0 1.0 
2.0 0.0 0.0 
-----------
[3]
2.0 2.0 1.0 
1.0 2.0 1.0 
2.0 0.0 0.0 
-----------
[3]
2.0 2.0 1.0 
1.0 2.0 1.0 
2.0 0.0 0.0 
-----------
[2]
2.0 2.0 1.0 
1.0 2.0 1.0 
2.0 0.0 0.0 
-----------
[1]
2.0 2.0 1.0 
1.0 2.0 1.0 
2.0 0.0 0.0 
-----------
[7]
2.0 2.0 1.0 
1.0 2.0 1.0 
2.0 1.0 2.0 
-----------
[7]
0.0 0.0 0.0 
0.0 0.0 0.0 
2.0 1.0 0.0 
-----------
[3]
0.0 0.0 0.0 
1.0 2.0 0.0 
2.0 1.0 0.0 
-----------
[7]
0.0 0.0 0.0 
1.0 2.0 0.0 
2.0 1.0 0.0 
-----------
[3]
0.0 0.0 0.0 
1.0 2.0 0.0 
2.0 1.0 0.0 
-----------
[1]
0.0 1.0 0.0 
1.0 2.0 0.0 
2.0 1.0 2.0 
-----------
[3]
0.0 1.0 0.0 
1.0 2.0 0.0 
2.0 1.0 2.0 
-----------
[0]
1.0 1.0 2.0 
1.0 2.0 0.0 
2.0 1.0 2.0 
-----------
[0]
1.0 1.0 2.0 
1.0 2.0 0.0 
2.0 1.0 2.0 
-----------
[1]
1.0 1.

Burada, daha önceki gibi boş bir tahta üzerinde bir gözlem yapıyoruz çünkü modeli grafik oyununa aktarmak ve kullanmak için kullanacağımız yol bu.

In [8]:
env=MyEnv()
#obs = #env.reset()
obs = [0,0,0,0,0,0,0,0,0]
for i in range(6):
    action, _states = model.predict(obs)
    print(action)
    obs, rewards, dones, info = env.step(action)
    env.render(mode='console')
    #print(dones)
    if dones:
        obs = env.reset()
    print("-----------")

0
1 2 0 
0 0 0 
0 0 0 
-----------
[4]
1 2 0 
2 1 0 
0 0 0 
-----------
[3]
1 2 0 
2 1 0 
0 0 0 
-----------
[5]
1 2 2 
2 1 1 
0 0 0 
-----------
[0]
1 2 2 
2 1 1 
0 0 0 
-----------
[4]
1 2 2 
2 1 1 
0 0 0 
-----------


/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)
